# Copy Data from a URL to Filecoin

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bacalhau-project/examples/blob/main/data-ingestion/from-url/index.ipynb)
[![Open In Binder](https://mybinder.org/badge.svg)](https://mybinder.org/v2/gh/bacalhau-project/examples/HEAD?labpath=data-ingestion/from-url/index.ipynb)

This notebook shows how to use Bacalhau to copy data from a URL to Filecoin and expose on IPFS for use with Bacalhau.

This takes advantage of the fact that all Bacalhau jobs are published to a Filecoin contract via Estuary. All data that is located in the `/outputs` directory is published.

The example below uses a simple tool we have created to help make it easier to move data in Bacalhau.

### Prerequisites

* [The Bacalhau client](https://docs.bacalhau.org/getting-started/installation)
* [`jq` to parse the Bacalhau output](https://stedolan.github.io/jq/download/)

In [2]:
!command -v bacalhau >/dev/null 2>&1 || (export BACALHAU_INSTALL_DIR=.; curl -sL https://get.bacalhau.org/install.sh | bash)
path=!echo $PATH
%env PATH=./:{path[-1]}
!command -v jq >/dev/null 2>&1 || (sudo apt update && sudo apt install -y jq)

/Users/phil/.zshenv:.:1: no such file or directory: /Users/phil/.cargo/env
env: PATH=./:./:/Users/phil/.pyenv/versions/3.9.7/bin:/opt/homebrew/Caskroom/google-cloud-sdk/latest/google-cloud-sdk/bin:/Users/phil/.gvm/bin:/opt/homebrew/opt/findutils/libexec/gnubin:/opt/homebrew/opt/coreutils/libexec/gnubin:/opt/homebrew/Caskroom/google-cloud-sdk/latest/google-cloud-sdk/bin:/Users/phil/.pyenv/shims:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/usr/local/MacGPG2/bin:/Users/phil/source/bacalhau-project/examples/workload-onboarding/rust-wasm/./bin:/Users/phil/.nexustools
/Users/phil/.zshenv:.:1: no such file or directory: /Users/phil/.cargo/env


## Uploading A File From a URL

To upload a file from a URL we will take advantage of the `--input-urls` parameter of the `bacalhau docker run` command. This will download a file from a public URL and place it in the `/inputs` directory of the container (by default).

Then we will use a helper container to move that data to the `/outputs` directory so that it is published to Filecoin via Estuary.

:::tip
You can find out more about the [helper container in the examples repository](https://github.com/bacalhau-project/examples/tree/main/tools/upload).
:::

In [8]:
%%bash --out job_id
bacalhau docker run \
    --id-only \
    --wait \
    --input-urls=https://raw.githubusercontent.com/filecoin-project/bacalhau/main/README.md \
    ghcr.io/bacalhau-project/examples/upload:v1

In [11]:
%env JOB_ID={job_id}

env: JOB_ID=f1725e46-141b-4e58-8b35-a62d85068ff1


Just to be safe, double check that the job succeeded by running the describe command (and some `jq` to parse it).

In [34]:
%%bash
bacalhau list $JOB_ID --output=json | jq '.[0].JobState.Nodes[] | .Shards."0" | select(.RunOutput)'

{
  "NodeId": "QmVAb7r2pKWCuyLpYWoZr9syhhFnTWeFaByHdb8PkkhLQG",
  "State": "Completed",
  "Status": "Got results proposal of length: 0",
  "VerificationResult": {
    "Complete": true,
    "Result": true
  },
  "PublishedResults": {
    "StorageSource": "IPFS",
    "Name": "job-f1725e46-141b-4e58-8b35-a62d85068ff1-shard-0-host-QmVAb7r2pKWCuyLpYWoZr9syhhFnTWeFaByHdb8PkkhLQG",
    "CID": "QmVkwhgH9j8MrjrWWv2SBXAD66m9DDpq4fMHs49seLGKWS"
  },
  "RunOutput": {
    "stdout": "1:54PM INF Copying files InputPath=/inputs OutputPath=/outputs\n1:54PM INF Copying object dst=/outputs/README.md src=/inputs/README.md\n1:54PM INF Done copying all objects files=[\"/outputs\",\"/outputs/README.md\"]\n",
    "stdouttruncated": false,
    "stderr": "",
    "stderrtruncated": false,
    "exitCode": 0,
    "runnerError": ""
  }
}


## Get the CID From the Completed Job

The job will upload the CID to the Filecoin network via Estuary. Let's get the CID from the output.

In [46]:
%%bash --out cid
bacalhau list $JOB_ID --output=json | jq -r '.[0].JobState.Nodes[] | .Shards."0".PublishedResults | select(.CID) | .CID'

In [47]:
%env CID={cid}

env: CID=QmVkwhgH9j8MrjrWWv2SBXAD66m9DDpq4fMHs49seLGKWS


In [55]:
from IPython.core.display import display, HTML
display(HTML('Your CID is <b>' + cid + '.</b><br/><a href="https://ipfs.io/ipfs/' + cid + '"><button>View files on ipfs.io</button></a>'))

## Use the CID in a New Bacalhau Job

Now that we have the CID, we can use it in a new job. This time we will use the `--inputs` parameter to tell Bacalhau to use the CID we just uploaded.

In this case my "job" is just to list the contents of the `/inputs` directory and cat the file we downloaded in the first step. You can see that the "input" data is located under `/inputs/outputs/README.md`.

In [63]:
%%bash --out job_id
bacalhau docker run \
    --id-only \
    --wait \
    --inputs=$CID \
    ubuntu -- \
    bash -c "set -x; ls -l /inputs; ls -l /inputs/outputs; cat /inputs/outputs/README.md"

In [64]:
%env JOB_ID={job_id}

env: JOB_ID=0089acb8-e9c6-414c-ba86-8b4bd8a02361


In [65]:
%%bash
rm -rf results && mkdir ./results
bacalhau get --output-dir ./results $JOB_ID 

Fetching results of job '0089acb8-e9c6-414c-ba86-8b4bd8a02361'...
Results for job '0089acb8-e9c6-414c-ba86-8b4bd8a02361' have been written to...
./results


In [70]:
%cat ./results/combined_results/stdout

/Users/phil/.zshenv:.:1: no such file or directory: /Users/phil/.cargo/env
total 12
-rw-rw-r-- 1 1000 1000    1 Dec  8 14:36 exitCode
drwxrwxr-x 2 1000 1000 4096 Dec  8 14:36 outputs
-rw-rw-r-- 1 1000 1000    0 Dec  8 14:36 stderr
-rw-rw-r-- 1 1000 1000  210 Dec  8 14:36 stdout
total 4
-rw-rw-r-- 1 1000 1000 3802 Dec  8 14:36 README.md
<!-- commenting out until we can fix the image logo [![CircleCI](https://dl.circleci.com/status-badge/img/null/filecoin-project/bacalhau/tree/main.svg?style=svg)](https://dl.circleci.com/status-badge/redirect/null/filecoin-project/bacalhau/tree/main)
-->
# The Filecoin Distributed Computation Framework  
<p align="center">
  <img src="docs/images/bacalhau-fish.jpg" alt="Bacalhau Logo" width="400" />
</p>
<p align=center>
  Compute Over Data == CoD
  <br>
  Bacalhau == "Salted CoD Fish" (Portuguese)
</p>
  
<br>

The purpose of Bacalhau is to provide a platform for public, transparent, and optionally verifiable computation. Bacalhau enables users to run a